In [ ]:
import csv
import pandas as pd
import urllib3
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Disable the urllib3 warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create a geolocator object with rate limiting
geolocator = Nominatim(user_agent="my_app")
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Read the input CSV file into a pandas DataFrame
input_df = pd.read_excel('/content/ForZipCodes-Chicago-Remaining.xlsx')

# Set the maximum number of rows per output file
max_rows_per_file = 500

# Initialize the row counter and output file counter
row_count = 0
file_count = 1

# Loop through each row in the input DataFrame
for index, row in input_df.iterrows():
    # Extract the latitude and longitude from the row
    lat = row['Latitude']
    lon = row['Longitude']

    # Use the geolocator to get the zip code from the latitude and longitude
    try:
        location = geocode(f"{lat}, {lon}", exactly_one=True, timeout=10)
        if location.raw['address'].get('postcode'):
            zip_code = location.raw['address']['postcode']
            row['Zipcode'] = zip_code
        else:
            row['Zipcode'] = "Zipcode not found"
    except (urllib3.exceptions.ConnectTimeoutError, urllib3.exceptions.ReadTimeoutError):
        row['Zipcode'] = "Timeout getting zip code"
    except Exception:
        row['Zipcode'] = "Error getting zip code"

    # Append the updated row to the output CSV file
    if row_count == 0:
        # If this is the first row in a new output file, create the file
        output_file_name = f"output_file_{file_count}.csv"
        output_df = pd.DataFrame(columns=input_df.columns)
        output_df.to_csv(output_file_name, index=False)

    # Append the updated row to the current output file
    output_df = pd.concat([output_df, row.to_frame().transpose()], ignore_index=True)
    output_df.to_csv(output_file_name, index=False)

    # Increment the row counter
    row_count += 1

    # If we've reached the maximum rows per output file, reset the row counter and increment the output file counter
    if row_count == max_rows_per_file:
        row_count = 0
        file_count += 1

    # Print a status update every 1000 rows
    if (index + 1) % 1000 == 0:
        print(f"Processed {index + 1} rows")

# If there are any remaining rows in the output DataFrame, write them to a new output file
if row_count > 0:
    output_file_name = f"output_file_{file_count}.csv"
    output_df.to_csv(output_file_name, index=False)


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.9/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.9/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py", line 1042, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.9/dist-packages/urllib3/con